In [2]:
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from generic_optimizer import GenericOptimizer
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import pickle
import os

from scipy.stats import ranksums

import re

from novel_sampling import over_sample, BiasedBinarySampling
from generic_optimizer import GenericOptimizer, prepare_splits

import matplotlib.pyplot as plt

In [3]:
baselines = {}
for folder in os.listdir('../Datasets'):
    
    x, y, splits = prepare_splits(os.path.join('../Datasets', folder, f"{folder}.csv"))
    iteration_values = []
    for train_idx, test_idx, validation_idx in splits: # 31 splits for Wilcoxon rank-sum
        x_train, y_train = x[train_idx], y[train_idx]
        x_test, y_test = x[test_idx], y[test_idx]

        baseline_error = GenericOptimizer.calculate_overall_error(
            x_train,
            y_train,
            x_test,
            y_test,
            GenericOptimizer.n_neighbours
        )

        iteration_values.append(baseline_error)
        
    baselines[folder] = iteration_values

C:\Users\RjKim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\RjKim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 556, in run
    w

In [4]:
execution_configs = []
path_string = "optimizations"
for file in os.listdir(path_string):
    with open(os.path.join(path_string, file), 'rb') as fh:
        execution_configs.append(pickle.load(fh))

In [5]:
def calculate(execution_config):
    x_train = execution_config["x"][execution_config["train_idx"]]
    y_train = execution_config["y"][execution_config["train_idx"]]
    
    if execution_config["Over Sample"] == "over_sample": 
        x_train, y_train = over_sample(x_train, y_train)
    
    x_validation = execution_config["x"][execution_config["validation_idx"]]
    y_validation = execution_config["y"][execution_config["validation_idx"]]
    x_test = execution_config["x"][execution_config["test_idx"]]
    y_test = execution_config["y"][execution_config["test_idx"]]
    
    validation_metrics = execution_config["Result"].F
    optimization_population = execution_config["Result"].X
    print("")
    print(f"Name: {execution_config['Run Name']}")
    print(f"Sampling: {execution_config['Over Sample']}")
    print(f"Optimized pop: {optimization_population.shape}")
    print(f"x_train: {x_train.shape}")
    print(f"y_train: {y_train.shape}")
    validation_pareto_front, best_x_train, best_y_train = GenericOptimizer.calculate_optimal_instance(
        x_train,
        y_train,
        x_validation,
        y_validation,
        validation_metrics,
        optimization_population,
        GenericOptimizer.n_neighbours
    )
    execution_config["Best X"] = best_x_train
    execution_config["Best Y"] = best_y_train
    optimized_test_err = GenericOptimizer.calculate_overall_error(
        best_x_train,
        best_y_train,
        x_test,
        y_test,
        GenericOptimizer.n_neighbours
    )
    # optimized_validation_err = GenericOptimizer.calculate_overall_error(
    #     best_x_train,
    #     best_y_train,
    #     x_validation,
    #     y_validation,
    #     GenericOptimizer.n_neighbours
    # )
    
    if False:
        test_metrics = []
        for objective in experiment_config['objectives_list']:
            test_metrics.append(GenericOptimizer.unbound_eval_objectives(
                objective,
                optimization_population[validation_pareto_front],
                x_train, y_train,
                x_test, y_test
            ))
        test_metrics = np.column_stack(test_metrics)
        test_pareto_front, best_x_TEST, best_y_TEST = GenericOptimizer.calculate_optimal_instance(
            x_train,
            y_train,
            x_test,
            y_test,
            test_metrics,
            optimization_population[validation_pareto_front],
            GenericOptimizer.n_neighbours
        )
        rank_one_pareto_front_metrics = test_metrics[test_pareto_front]
        x = rank_one_pareto_front_metrics[:,0]
        y = rank_one_pareto_front_metrics[:,1]
        x1 = validation_metrics[validation_pareto_front][:,0]
        y1 = validation_metrics[validation_pareto_front][:,1]
        plt.scatter(x, y)
        plt.scatter(x1, y1)
        plt.xlim([0, 1])
        plt.show()
        print(metrics)
        print(fronts)

    return execution_config, optimized_test_err

redo = []
output = []
for execution_config in execution_configs:
    try:
        output.append(calculate(execution_config))
    except:
        redo.append(execution_config)
        
# output = Parallel(n_jobs=-1)(delayed(calculate)(execution_config) for execution_config in execution_configs)
with open("calc_dump.pickle", "wb") as fh:
    pickle.dump(output, fh)


Name: 0_over_sample_bias_population_australian_calculate_class0_error_calculate_class1_error
Sampling: over_sample
Optimized pop: (100, 537)
x_train: (537, 14)
y_train: (537,)

Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False


Name: 0_over_sample_bias_population_australian_calculate_overall_error_calculate_num_examples
Sampling: over_sample
Optimized pop: (9, 536)
x_train: (536, 14)
y_train: (536,)

Name: 0_over_sample_bias_population_australian_calculate_overall_error_calculate_overall_inverse_f1_calculate_num_examples
Sampling: over_sample
Optimized pop: (27, 536)
x_train: (536, 14)
y_train: (536,)

Name: 0_over_sample_bias_population_bupa_calculate_class0_error_calculate_class1_error
Sampling: over_sample
Optimized pop: (100, 272)
x_train: (272, 6)
y_train: (272,)

Name: 0_over_sample_bias_population_bupa_calculate_overall_erro

In [6]:
len(redo)

0

In [ ]:
with open("calc_dump.pickle", "rb") as fh:
    output = pickle.load(fh)

In [7]:
aggregated_results = {}
for config, optimized_error in output:
    execution_type = config["Dataset"] + "<>" + "_".join(config["Run Name"].split("_")[1:])

    if execution_type not in aggregated_results:
        aggregated_results[execution_type] = []

    aggregated_results[execution_type].append(optimized_error) 


In [8]:
for key in aggregated_results:
    dataset = key.split("<>")[0]
    optimized_distribution = aggregated_results[key]
    baseline_distribution = baselines[dataset]

    wilcoxon_test = ranksums(baseline_distribution, optimized_distribution, alternative='less')

    mean = np.mean(optimized_distribution)
    median = np.median(optimized_distribution)
    std_dev = np.std(optimized_distribution)
    min_value = np.min(optimized_distribution)
    max_value = np.max(optimized_distribution)
    variance = np.var(optimized_distribution)

    if wilcoxon_test.pvalue > 0.05:
        print(key.split("<>")[1])
        print(f"mean: {mean}")
        print(f"median: {median}")
        print(f"std_dev: {std_dev}")
        print(f"min_value: {min_value}")
        print(f"max_value: {max_value}")
        print(f"variance: {variance}")
        print(f"P-val: {wilcoxon_test.pvalue}")

        print("--------------------------------\n")


over_sample_bias_population_australian_calculate_class0_error_calculate_class1_error
mean: 0.35353347007272046
median: 0.35838150289017345
std_dev: 0.028849356760697754
min_value: 0.3063583815028902
max_value: 0.4104046242774566
variance: 0.0008322853855060173
P-val: 0.316086171555764
--------------------------------

over_sample_bias_population_australian_calculate_overall_error_calculate_num_examples
mean: 0.35782211448815965
median: 0.35838150289017345
std_dev: 0.029156247941659302
min_value: 0.3063583815028902
max_value: 0.4219653179190751
variance: 0.0008500867940355124
P-val: 0.19329107378408628
--------------------------------

over_sample_bias_population_australian_calculate_overall_error_calculate_overall_inverse_f1_calculate_num_examples
mean: 0.3537199328733917
median: 0.36416184971098264
std_dev: 0.03387778169963358
min_value: 0.2658959537572254
max_value: 0.4046242774566474
variance: 0.001147704092888028
P-val: 0.23855204375332778
--------------------------------

over_sam

In [4]:
execution_types = {}
for result_name, test_err in output:
    if result_name not in execution_types:
        execution_types[result_name] = []
    execution_types[result_name].append(test_err)

len(execution_types["over_sample_rand_population_australian_calculate_overall_error_calculate_num_examples"])


NameError: name 'output' is not defined

In [18]:
for conf in redo:
    print(conf["Run Name"])

0_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_sonar_calculate

In [ ]:
p